<h1> Lecture 16  

Data Science 8, Spring 2021 </h1>

<h3>
<b>
<ul>
<li>Models of Randomness (Chance)   </li><br>
            
<li>Is the model consistent with the Data?  </li><br>
</ul>
</b>
</h3>

In [ ]:
from datascience import *
import numpy as np
import warnings
warnings.filterwarnings("ignore")

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
plots.rcParams["patch.force_edgecolor"] = True

#The following allows porting images into a Markdown window
from IPython.display import Image

<h2> Large Random Samples</h2>

<h3>Back to our United Airlines Flight Data </h3>

In [ ]:
united = Table.read_table('united.csv')
row_numbers = np.arange(united.num_rows)
#row_numbers
united = united.with_column('Row', row_numbers)
united = united.move_column('Row',0)
united

<h4> Alternative way of adding a Row column and moving it the the leftmost column. </h4>

In [ ]:
united = Table.read_table('united.csv')
row_numbers = np.arange(united.num_rows)
united = united.with_column('Row', row_numbers).move_to_start('Row')
united

<h4>Create histogram bins</h4>

In [ ]:
united_bins = np.arange(-20, 301, 5)

<h3>The histogram below is the <i>actual</i> histogram from the entire data set of flights.  We can think of it as the probability distribution for the population (population of flights, that is).<br>
    
This is equivalent to  having access to the theoretical probability distribution underlying the data.<br>

In the real world, typically we can <i>not</i> sample the entire population, so we do <i>not</i> have access to such a comprehensive distribution.
</h3>

In [ ]:
# (Population) Probability Distribution
united.hist('Delay', bins = united_bins)

<h3> Construct empirical distributions using 10, 100, and 1,000 samples (i.e., flights) from the table.</h3><br>

<h4> If we run each of the three cells below multiple times, we're bound to see the different results.  This is because each time we sample a potentially different set of rows from the original table is selected.</h4>

<h4>Question: Which number of samples among 10, 100, and 1,000 produces a histogram that is more stable in overall shape?</h4>

In [ ]:
# (Sample) Empirical Distribution
united.sample(10).hist('Delay', bins = united_bins)

In [ ]:
# (Sample) Empirical Distribution
united.sample(100).hist('Delay', bins = united_bins)

In [ ]:
# (Sample) Empirical Distribution
united.sample(1000).hist('Delay', bins = united_bins)

<h2>Inference</h2>

SLIDE: Inference

## Statistics

<h2>Median:</h2>
<h3>
The "middle" of a sorted list of numerical values. Half the values are above and half are below median. <br>
<ul>
    <li>   $\{2,3,5,5,6,7\}$ </li><br>
    <li> Median is $5$.</li><br>
    <li> If there are two middle numbers, we typically average them.
</ul>
</h3>

<h3> Actual Median Delay in Our Data Set</h3>

In [ ]:
# (Population) Parameter
np.median(united.column('Delay'))

<h3>Median Delay Estimated from 10 Samples</h3>

<h4>Run this a few times to get a sense of the range of values.</h4>

In [ ]:
# (Sample) Statistic
np.median(united.sample(10).column('Delay'))

<h3>Median Delay Estimated from 100 Samples</h3>

<h4>Run this a few times to get a sense of the range of values.<br>

Is the range of values tighter than above?
</h4>

In [ ]:
# (Sample) Statistic
np.median(united.sample(100).column('Delay'))

<h3>Median Delay Estimated from 1,000 Samples</h3>

<h4>Run this a few times to get a sense of the range of values.<br>

Is the range of values tighter than above?
</h4>

In [ ]:
# (Sample) Statistic
np.median(united.sample(1000).column('Delay'))

### Probability & Empirical Distributions of a Statistic

In [ ]:
def sample_median(size):
    return np.median(united.sample(size).column('Delay'))

<h3> Let's dissect the function <tt>sample_median</tt>
    <ul>
        <li> Take a random sample of <tt>size</tt> rows (e.g., <tt>size=10</tt> rows) from the United table.</li><br>
        <li> Grab the 'Delay' column from the table of <tt>size</tt> sample rows, and produce an array from the numbers in that column.</li><br>
        <li> Compute the median delay from the array of delays. 
    </ul>
</h3>

<h3>Let's dissect the code above</h3>

In [ ]:
size=10
my_sample=united.sample(size)
my_sample

<h4>Grab the numbers from the Delay column, and produce an array</h4>

In [ ]:
size=10
my_sample_delay_array = my_sample.column('Delay')
my_sample_delay_array

<h4>Compute the median delay sample of delays</h4>

In [ ]:
np.median(my_sample_delay_array)

<h4>Try rerunning the three cells above.  You should see different results each time.</h4>

<h3>Let's return to our <tt>sample_median</tt> function and run it to capture a sample size of 10.<br>
Try running the cell below several times. </h3>

In [ ]:
sample_median(10)

In [ ]:
num_simulations = 2000

In [ ]:
#Create an empty array to contain each run of the num_simulations of sample_median(size)
sample_medians = make_array()

for i in np.arange(num_simulations):
    new_median = sample_median(10)
    sample_medians = np.append(sample_medians, new_median)

<h3>Make a table with all the size-10 sample medians</h3>

In [ ]:
Table().with_column('Sample medians (size=10)', sample_medians)#.show()

<h3>Now plot the histogram of the size-10 sample medians</h3>

In [ ]:
Table().with_column('Sample medians (size=10)', sample_medians).hist(bins=20) #bins=20 asks Python to create 20 bins

<h3>Now change the sample size from 10 to 1,000,<br>

but still run <tt>num_simulations=2000</tt> trials.
</h3>

In [ ]:
sample_medians = make_array()

for i in np.arange(num_simulations):
    new_median = sample_median(1000)
    sample_medians = np.append(sample_medians, new_median)

In [ ]:
Table().with_column('Sample medians (size=1K)', sample_medians).hist()

<h3> Now Overlay All Three Empirical Distributions (for sample sizes 10, 100, and 1,000)</h3>

In [ ]:
sample_medians_10 = make_array()
sample_medians_100 = make_array()
sample_medians_1000 = make_array()

num_simulations = 2000

for i in np.arange(num_simulations):
    new_median_10 = sample_median(10)
    sample_medians_10 = np.append(sample_medians_10, new_median_10)
    new_median_100 = sample_median(100)
    sample_medians_100 = np.append(sample_medians_100, new_median_100)
    new_median_1000 = sample_median(1000)
    sample_medians_1000 = np.append(sample_medians_1000, new_median_1000)

In [ ]:
sample_medians = Table().with_columns('Size 10', sample_medians_10, 
                                      'Size 100', sample_medians_100,
                                      'Size 1000', sample_medians_1000)
sample_medians

<h3> Minimum Delays for Size 10, Size 100, and Size 1,000 Samples</h3>

In [ ]:
[min(sample_medians.column('Size 10')),
 min(sample_medians.column('Size 100')),
 min(sample_medians.column('Size 1000'))]

<h3> Maximum Delays for Size 10, Size 100, and Size 1,000 Samples</h3>

In [ ]:
[max(sample_medians.column('Size 10')),
 max(sample_medians.column('Size 100')),
 max(sample_medians.column('Size 1000'))]

In [ ]:
sample_medians.hist(bins = np.arange(-5, 46))

<h3> As expected, the tightest histogram belongs to the largest sample size.<br>
    <ul>
        <li> These are empirical distributions of a statistic (in this case, the statistic is the sample median).</li><br>
        <li> Law of Large Numbers Applied to the Statistic (in this case, the median): As the sample size increases, the empirical distributions of a statistic get closer and closer to the actual value that the statistic is estimating (in this case, the sample median is an approximation to the true median of the entire population of nearly 14,000 flights). The true median (for the entire population of flights) was found to be 2 mins.</li><br>
        <li> Law of Large Numbers Applied to the Data (in this case, the delays): The empirical distributions of the samples of date approach the true probability distribution of the entire population, as the sample size increases.        
    </ul>
    
    
</h3>

SLIDE: Assessing Models

## Swain vs. Alabama ##

<h3>Make an array that contains the population proportions: 26% Black, and 74% White.</h3>

In [ ]:
population_proportions = make_array(.26, .74)
population_proportions

<h3> Now we take 100 samples from the population. </h3>

In [ ]:
sample_proportions(100, population_proportions)

<h4>Run the cell above many times to see how the numbers change, if by much. </h4>

In [ ]:
def panel_proportion():
    return sample_proportions(100, population_proportions)

In [ ]:
panel_proportion()

<h3>We need only the first number in the array&mdash;that is, the fraction of black jurors in the sample.</h3>

In [ ]:
def panel_proportion():
    return sample_proportions(100, population_proportions).item(0) 
#item(0) returns the fraction, of the sample population, that are Black.

In [ ]:
panel_proportion()

In [ ]:
#Specify the number of trials---that is, the number of times we take a sample
num_simulations = 1000

panels = make_array()

for i in np.arange(num_simulations):
    new_panel = panel_proportion() * 100 #Multiplication by 100 converts the fraction to actual headcounts
    panels = np.append(panels, new_panel)
#panels

In [ ]:
Table().with_column('Number of Black Men on Panel of 100', panels).hist(bins=np.arange(5.5,40.))

SLIDE: Genetic Model and Gregor Mendel

## Mendel and Pea Flowers ##

In [ ]:
## Mendel had 929 plants, of which 709 had purple flowers
observed_purples = 709 / 929
observed_purples

In [ ]:
predicted_proportions = make_array(.75, .25)
sample_proportions(929, predicted_proportions)

In [ ]:
def purple_flowers():
    return sample_proportions(929, predicted_proportions).item(0) * 100

In [ ]:
purple_flowers()

In [ ]:
num_simulations=1000

purples = make_array()

for i in np.arange(num_simulations):
    new_purple = purple_flowers()
    purples = np.append(purples, new_purple)

In [ ]:
Table().with_column('Percent of purple flowers in sample of 929', purples).hist()

In [ ]:
Table().with_column('Discrepancy in sample of 929 if the model is true', abs(purples- 75)).hist()

<h3>How close was Mendel's sample to the model's prediction?</h3>

In [ ]:
abs(observed_purples * 100 - 75)

SLIDE: Model and Alternative